# Toy example for Radix-based for multiple conditions

In [1]:
SCALE_CROWDING=False
PREVENT_ZERO = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7fc2f1d6fef0>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
mdl_ref = ijomc
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound0 = 0.001
crowding_bound = crowding_bound0

not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [7]:
N_CONDS = 2

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [8]:
stacker = StackOptimizer()
stacker.stack_models(ijomc, df_conds)

In [9]:
stacker.model.optimize(solver='gurobi')

<Solution 1.75 at 0x7fc291924b90>

In [10]:
stacker.model_dict['fructose'].optimize()

<Solution 0.87 at 0x7fc291924ad0>

In [11]:
stacker.model_dict['glucose'].optimize()
print stacker.model_dict['glucose'].reactions.query('BIOMASS')[0].x
print stacker.model_dict['glucose'].reactions.query('EX_glc')[0].x
print stacker.model_dict['glucose'].reactions.query('EX_fru')[0].x

0.873921506968
-10.0
0.0


In [12]:
stacker.model_dict['fructose'].optimize()
print stacker.model_dict['fructose'].reactions.query('BIOMASS')[0].x
print stacker.model_dict['fructose'].reactions.query('EX_glc')[0].x
print stacker.model_dict['fructose'].reactions.query('EX_fru')[0].x

0.873921506966
0.0
-10.0


In [13]:
for mdl_ind,mdl in iteritems(stacker.model_dict):
    opt = Optimizer(mdl)
    gapi = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)

In [14]:
stacker.model.optimize(solver='gurobi')

<Solution 3.50 at 0x7fc291aa4890>

In [15]:
import numpy as np

radix = 2.
print('Radix:',radix)
powers = np.arange(-3,4)
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Discretize crowding coefficients into radix
#crowding_p = gap.metabolites.crowding
#crowding_cons = stacker.model.metabolites.query('crowding')

var_cons_dict = {}
# Get the group ID from reference model
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
for rxn_ref in crowding_ref.reactions:    
    for mdl_ind, mdl in iteritems(stacker.model_dict):
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        # Get the coefficient in the dual
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

('Radix:', 2.0)
('Powers:', array([-3, -2, -1,  0,  1,  2,  3]))
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [16]:
# %lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power, prevent_zero=True)

In [17]:
%time opt.to_radix(stacker.model, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)

CPU times: user 115 ms, sys: 17.5 ms, total: 133 ms
Wall time: 117 ms


[1.0]

### Try scaling
if SCALE_CROWDING:
    crowding = stacker.model.metabolites.crowding
    bound0  = crowding._bound
    f_scale = crowding._bound
    crowding._bound = bound0 / f_scale
    for rxn in crowding.reactions:
        rxn._metabolites[crowding] = rxn._metabolites[crowding]/f_scale

In [18]:
for rxn in stacker.model.reactions:
    rxn.objective_coefficient = 0.
for rxn in stacker.model.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [22]:
from cobra.solvers import gurobi_solver
from gurobipy import *

F_TOL_FINAL = 0.02
F_TOL_START = 1.
N_FS =11
F_TOLS = np.linspace(F_TOL_START, F_TOL_FINAL, N_FS)
F_TOLS

array([1.   , 0.902, 0.804, 0.706, 0.608, 0.51 , 0.412, 0.314, 0.216,
       0.118, 0.02 ])

### TOOD: bisection on F_TOL instead of linear

In [23]:
%%time

mu_id = 'BIOMASS_Ecoli_core_w_GAM'
sol_dict = {}
for ind, F_TOL in enumerate(F_TOLS):
    for mdl_ind,mdl in iteritems(stacker.model_dict):
        mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
        rxn_mu = mdl.reactions.get_by_id(mu_id+'_%s'%mdl_ind)    
        rxn_mu.lower_bound = mu_meas*(1-F_TOL)
        rxn_mu.upper_bound = mu_meas*(1+F_TOL)
        mu_crowd0 = np.nan
        print('Condition=%s. Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mdl_ind, mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))

    # If feasible solution found, use it as MILP start
    if milp.SolCount > 0:
        print('#'*40)
        print('Setting MIP warm-start values.')
        for v in milp.getVars():
            v.Start = v.X
        sol_dict[ind] = {'x_dict':{v.VarName:v.X for v in milp.getVars()}, 'F_TOL':F_TOL}

    milp = gurobi_solver.create_problem(stacker.model)
    milp.ModelSense = GRB.MINIMIZE
    milp.Params.IntFeasTol = 1e-9
    milp.Params.OutputFlag = 1
    # milp.Params.FeasibilityTol = 1e-9
    # milp.Params.OptimalityTol = 1e-9
    milp.Params.NodefileStart = 3   # Start writing nodes to disk if x GB RAM exceeded (for each thread)
    milp.Params.TimeLimit = 2*3600  # Time limit in seconds

    milp.optimize()

Condition=fructose. Initial mu_crowd=nan. Fitting within 100% of measured: 0 <= mu <= 1.1
Condition=glucose. Initial mu_crowd=nan. Fitting within 100% of measured: 0 <= mu <= 1.48


NameError: name 'milp' is not defined

In [24]:
[(k, s['F_TOL']) for k,s in iteritems(sol_dict)]

[]

In [ ]:
len(sol_dict[9]['x_dict'])

In [ ]:
len(stacker.model.reactions)

In [ ]:
best_ind = len(sol_dict)
best_ind

In [ ]:
x_dict = {r.id:sol_dict[best_ind]['x_dict'][str(i)] for i,r in enumerate(stacker.model.reactions)}
F_TOL = sol_dict[best_ind]['F_TOL']

In [ ]:
df_meas.head()

In [ ]:
#sol = gurobi_solver.format_solution(milp, stacker.model)
#yopt = [sol.x_dict[rxn.id] for rxn in stacker.model.reactions.query('binary_')]
yopt = [x_dict[rxn.id] for rxn in stacker.model.reactions.query('binary_')]
print('Number of non-zero binaries: %g' % sum(yopt))
for mdl_ind,mdl in iteritems(stacker.model_dict):
    dfi = df_meas[ df_meas.substrate==mdl_ind]
    mu_measi = dfi.growth_rate_1_h.iloc[0]
    rxn_mu = stacker.model.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    muopt = x_dict[rxn_mu.id]
    muL = mu_measi*(1-F_TOL)
    muU = mu_measi*(1+F_TOL)
    print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, muL, muopt, muU))

In [ ]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = x_dict[yid]            
            if abs(y)>1e-10:
                print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
#                 print yid, y
#             if abs(y)<1e-10:
#                 print yid, y
                pass
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

In [ ]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

### Plug back in to be sure

In [ ]:
df_conds

In [ ]:
#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:
    ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')    
    crowding = Constraint('crowding')
    crowding._bound = crowding_bound0
    crowding._constraint_sense = 'L'
    
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit})
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    
    print('Cond=%s. mu_meas=%g. mu_sim=%g' % (csrc, mu_measi, ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x))    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))